In [1]:
import pandas as pd
import numpy as np

In [ ]:
## Steps to perform:

### 1.  Create RFM analysis:
### ### A) Create Cohort_Analysis
### ### ### ### Cohort_Month(funkcja + kolumna (1 dzien miesiaca))
### ### ### ### Cohort_Index (od 1 kohorty do ostatniej)
### ### ### ### Cohort HeatMap



### ### B) Calculate RFM values
### ### C) Assign RFM classes as per values (from finger or Clustering)



### 2.  Analyze data as per 'returnQuantity'
### 3.  Perform Feature Engineering
### ### A) RETURN two strategies: (1: if return 2: how big return) vs (poisson / other rozklad)
### ### B) Analyses to be created:
### ### ### ### 1. Customer analyzes (how many categories per purchase, how many in total,
### ### ### ### ### ### ### ### spend% per category, how often vouchers (out of all visits)
### ### ### ### ### ### ### ### vouchered_amount / total_spend, how many products per purchase, how many items per basket.
### ### ### ### 2. Basket analyzes (how many categories in basket,
### ### ### ### ### ### ### ###
### ### ### ### 3. Products mixes (calculating substitutity/complementary): co z czym się kupuje....
### ### ### ### ### ### ### ###
## Questions to answer:
#### If buying more the same items but in different sizes/color (to match size)
#### If buying more the same category in basket: increase prob or return
### ### C) Possible variabales in Feat. Eng.:
### ### ### ### Discount Perctg (price / recom)
### ### ### ### If this purchase has 1 or few categories
### ### ### ### If this customer buys 1 or few categories
### ### ### ### If any products mix available( with top prob of return)
### 4.  Modeling:
### ### A) Creating basic model: very very basic (random or linear regression with basic variables (ininital dataset))
### ### B) Adding new features to model (FE, etc, one-hot-encoding)
### ### C) LR, DTr, RF, etc, Deep Learning

In [2]:
train = pd.read_csv('orders_train.txt', sep=';')

In [3]:
train.columns

Index(['orderID', 'orderDate', 'articleID', 'colorCode', 'sizeCode',
       'productGroup', 'quantity', 'price', 'rrp', 'voucherID',
       'voucherAmount', 'customerID', 'deviceID', 'paymentMethod',
       'returnQuantity'],
      dtype='object')

In [11]:
imp_col = ['orderID', 'articleID','voucherID','customerID']
for each in imp_col:
    print(train[each].nunique(),"\t\t",each)

738698 		 orderID
3823 		 articleID
670 		 voucherID
311369 		 customerID


In [12]:
train['orderDate'].min(), train['orderDate'].max()

('2014-01-01', '2015-09-30')

In [13]:
train.sample(5)

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,customerID,deviceID,paymentMethod,returnQuantity
525561,a1168536,2014-06-05,i1000376,1001,42,3.0,1,19.99,19.99,0,0.0,c1149538,2,BPRG,0
1063686,a1338769,2014-10-29,i1000682,3493,38,3.0,1,18.00,35.99,0,0.0,c1202983,2,BPRG,0
209805,a1069796,2014-03-15,i1002039,3002,44,8.0,1,39.99,39.99,0,0.0,c1020770,2,VORAUS,0
1612150,a1522495,2015-04-14,i1000323,3088,44,3.0,1,29.99,29.99,0,0.0,c1092444,2,BPRG,1
832429,a1261775,2014-08-21,i1001402,1001,36,5.0,1,49.99,49.99,0,0.0,c1007494,2,KKE,1


In [74]:
def calc_association(data,var = 'productGroup'):
    grouping = data[['productGroup','articleID','orderID','quantity']].groupby(['orderID',var]).quantity.sum().reset_index()
    grouping.columns = ['orderID',var,'check']
    grouping = grouping.sort_values('check', ascending = False)
    grouping['check'] = 1
    
    if var == 'articleID':
        grouping = grouping.head(500)
    
    group_piv = grouping.pivot(columns=grouping.columns[1],index='orderID',values='check').fillna(0)
    prod = round(pd.DataFrame(np.dot(group_piv.iloc[:,1:].T,group_piv.iloc[:,1:]))/group_piv.shape[0],4)
    
    table = []
    
    i = 0
    
    for c in range(len(prod.columns)):
        for r in range(c,len(prod.columns)):
            
            prod_aa = group_piv.columns[r]
            prod_bb = group_piv.columns[c]
            prob_aa = prod.iloc[r,r]
            prob_ab = prod.iloc[r,c]
            prob_bb = prod.iloc[c,c]
            
            likeliness = prob_ab / prob_aa
            association = prob_ab / (prob_aa * prob_bb)
    
            table.append([prod_aa,prod_bb,prob_aa,prob_ab,prob_bb,likeliness,association])
        
            tab1 = pd.DataFrame(table)
            tab1.columns = ['prod_aa','prod_bb','prob_aa','prob_ab','prob_bb','likeliness','association']
            tab1 = tab1[tab1['prod_aa'] != tab1['prod_bb']]
            tab2 = tab1.sort_values('association', ascending= False)
    return tab2.style.format({'prob_aa':'{:.2%}','prob_ab':'{:.2%}','prob_bb':'{:.2%}','likeliness':'{:.2%}','association':'{:.2}'})

In [72]:
calc_association(train,var='productGroup')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars


,prod_aa,prod_bb,prob_aa,prob_ab,prob_bb,likeliness,association
145,45,26,0.96%,0.01%,0.39%,1.04%,2.7
128,26,14,0.39%,0.01%,1.22%,2.56%,2.1
99,9,8,8.87%,1.22%,6.87%,13.75%,2.0
80,14,6,1.22%,0.10%,4.41%,8.20%,1.9
123,45,13,0.96%,0.08%,4.86%,8.33%,1.7
34,4,3,8.28%,1.27%,10.25%,15.34%,1.5
68,14,5,1.22%,0.07%,3.87%,5.74%,1.5
135,43,15,1.66%,0.35%,14.48%,21.08%,1.5
35,5,3,3.87%,0.57%,10.25%,14.73%,1.4
56,15,4,14.48%,1.70%,8.28%,11.74%,1.4


In [ ]:
calc_association(train,var='articleID')

In [36]:
train[(train['returnQuantity']>1)&(train['quantity']!=train['returnQuantity'])]

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,customerID,deviceID,paymentMethod,returnQuantity,count
4307,a1001513,2014-01-03,i1000526,2976,36,3.0,5,100.00,29.99,0,0.0,c1089966,3,CBA,4,1
5785,a1002010,2014-01-04,i1000312,1000,38,3.0,5,89.95,17.99,0,0.0,c1090173,2,PAYPALVC,2,0
32624,a1010973,2014-01-14,i1000321,1493,38,3.0,4,103.96,25.99,0,0.0,c1093258,2,BPRG,2,2
37902,a1012696,2014-01-15,i1000526,2976,42,3.0,3,60.00,29.99,0,0.0,c1093834,2,BPRG,2,0
42892,a1014236,2014-01-17,i1000325,1000,40,3.0,3,59.97,19.99,v1000078,10.0,c1044371,2,BPRG,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280187,a1729833,2015-09-18,i1002605,1082,I,17.0,4,134.44,39.99,0,0.0,c1283475,3,BPLS,3,51
2280188,a1729833,2015-09-18,i1002611,3111,I,17.0,5,117.60,27.99,0,0.0,c1283475,3,BPLS,2,52
2288881,a1732544,2015-09-21,i1002953,2089,I,17.0,3,21.00,22.99,v1000737,10.0,c1316465,2,BPRG,2,11
2290518,a1733111,2015-09-21,i1000397,1111,42,3.0,3,53.97,17.99,0,0.0,c1055271,3,BPRG,2,72


In [37]:
train.sizeCode.value_counts()

38     395163
40     393547
42     393244
44     359473
36     270462
34     147173
I      116374
A       42014
32      36554
29      30560
30      29115
31      26133
28      22634
33      20890
27      17139
26       9836
25       5104
85       1680
90       1613
95       1597
80       1157
100      1059
M         751
75        491
L         445
S         444
24        317
XL        158
XS         38
Name: sizeCode, dtype: int64

In [34]:
train['count'] = train.groupby(['customerID']).cumcount()
#train.sort_values(['customerID','orderDate'],ascending = [False,False])
first_order = train[train['count'] == 0][['customerID','orderDate']]
first_order.columns = ['customerID','f_order']
first_order.head()

,customerID,f_order
0,c1010575,2014-01-01
2,c1045905,2014-01-01
4,c1089295,2014-01-01
9,c1050116,2014-01-01
10,c1089296,2014-01-01


In [ ]:
pd.crosstab(train.paymentMethod, train.returnQuantity, normalize = 'index').round(3).sort_values(0,ascending = False)

In [14]:
pd.crosstab(train.productGroup, train.returnQuantity, normalize = 'index').round(3)

returnQuantity,0,1,2,3,4,5
productGroup,,,,,,
1.0,0.405,0.594,0.001,0.0,0.0,0.0
2.0,0.419,0.579,0.002,0.0,0.0,0.0
3.0,0.539,0.459,0.002,0.0,0.0,0.0
4.0,0.487,0.512,0.001,0.0,0.0,0.0
5.0,0.458,0.541,0.001,0.0,0.0,0.0
6.0,0.487,0.512,0.001,0.0,0.0,0.0
7.0,0.377,0.622,0.001,0.0,0.0,0.0
8.0,0.417,0.581,0.002,0.0,0.0,0.0
9.0,0.404,0.595,0.001,0.0,0.0,0.0


In [ ]:
# Calculate Recency, Frequency and Monetary value for each customer 
data_tran = f_tran.groupby(['user_id']).agg({
    'created_date': lambda x: (today - x.max()).days,
    'transaction_id': 'count',
    'amount_usd': 'sum'})

# Rename the columns 
data_tran.rename(columns={'created_date': 'Recency',
                         'transaction_id': 'Frequency',
                         'amount_usd': 'MonetaryValue'}, inplace=True)

# Print top 5 rows
print(data_tran.head())

In [7]:
train.shape

(2325165, 15)

In [17]:
final = pd.read_csv('orders_class.txt', sep=';')
final.head()

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,customerID,deviceID,paymentMethod
0,a1744178,2015-10-01,i1002632,3097,I,17.0,1,19.99,19.99,v1000415,10.0,c1062117,2,BPRG
1,a1744178,2015-10-01,i1003278,1097,40,3.0,1,12.95,12.95,v1000415,10.0,c1062117,2,BPRG
2,a1744178,2015-10-01,i1003279,1114,40,3.0,1,12.95,12.95,v1000415,10.0,c1062117,2,BPRG
3,a1744178,2015-10-01,i1003279,1116,40,3.0,1,12.95,12.95,v1000415,10.0,c1062117,2,BPRG
4,a1744178,2015-10-01,i1003279,1117,40,3.0,1,12.95,12.95,v1000415,10.0,c1062117,2,BPRG


In [18]:
final['orderDate'].min(), final['orderDate'].max()

('2015-10-01', '2015-12-31')

In [39]:
final.quantity.value_counts()

1    331590
0      6692
2      2512
3       211
5        52
4        41
Name: quantity, dtype: int64

## Train file

In [ ]:
# Create model

clf = DecisionTreeClassifier()

 

# Set up k-fold

k_fold = KFold(n_splits = 4, random_state = 0)

 

# Evaluate precision and recall for each fold

precision = cross_val_score(

  clf, X_train, y_train, cv = k_fold, scoring = 'precision_weighted')

recall = cross_val_score(

  clf, X_train, y_train, cv = k_fold, scoring = 'recall_weighted')

print("Precision scores: %s" %(precision))

print("Recall scores: %s" %(recall))